# Benchmark 1

We study the Hamiltonian of the Heisenberg model with periodic boundary conditions.

In [1]:
using CondensedMatterSOS
import MultivariatePolynomials
const MP = MultivariatePolynomials
@spin σ[1:3]
heisenberg_hamiltonian(σ, true)

# Let's pick a solver from [this list](https://jump.dev/JuMP.jl/dev/installation/#Getting-Solvers).

using CSDP
solver = optimizer_with_attributes(
    () -> MOIU.CachingOptimizer(MOIU.UniversalFallback(MOIU.Model{Float64}()), CSDP.Optimizer()),
    MOI.Silent() => false,
);

We can compute a lower bound `-2√2` to the ground state energy as follow:

In [2]:
function hamiltonian_energy(N, maxdegree, solver; symmetry=true, consecutive=false, kws...)
    @spin σ[1:N]
    H = heisenberg_hamiltonian(σ, true)
    G = Lattice1Group(N)
    cone = NonnegPolyInnerCone{SumOfSquares.COI.HermitianPositiveSemidefiniteConeTriangle}()
    @assert iseven(maxdegree)
    cert = SumOfSquares.Certificate.FixedBasis(
        cone,
        MonomialBasis(MP.monomials(σ[1], 0:div(maxdegree, 2), consecutive=consecutive)),
    )
    certificate = Symmetry.Ideal(
        Symmetry.Pattern(G, Action(σ)),
        cert,
    )
    if symmetry
        energy(H, maxdegree, solver; certificate = certificate, kws...)
    else
        energy(H, maxdegree, solver; kws...)
    end
end
bound, gram, ν = hamiltonian_energy(
    2,
    2,
    solver,
    symmetry = false,
    sparsity = SumOfSquares.Sparsity.NoPattern(),
)
bound

Iter: 15 Ap: 9.59e-01 Pobj: -1.2000000e+01 Ad: 9.60e-01 Dobj: -1.2000000e+01 
Success: SDP solved
Primal objective value: -1.2000000e+01 
Dual objective value: -1.2000000e+01 
Relative primal infeasibility: 2.20e-13 
Relative dual infeasibility: 3.91e-09 
Real Relative Gap: 3.71e-09 
XZ Relative Gap: 5.51e-09 
DIMACS error measures: 5.76e-13 0.00e+00 2.33e-08 0.00e+00 3.71e-09 5.51e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -5.6411147e+01 Ad: 4.06e-01 Dobj: -4.2832684e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -5.4760276e+01 Ad: 9.64e-01 Dobj: -7.6705709e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -2.1723633e+01 Ad: 8.78e-01 Dobj: -1.3767694e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -8.4915625e+00 Ad: 7.01e-01 Dobj: -3.2534741e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -6.6196805e+00 Ad: 8.87e-01 Dobj: -5.5119401e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -6.0450487e+00 Ad: 9.46e-01 Dobj: -5.9459371e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -6.0023373e+00 Ad: 9.58e-0

-6.000000022862929

We can see that the moment matrix uses all monomials:

In [3]:
ν.basis.monomials

7-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂

# Symmetry reduction

We can reduce the computation using symmetry reduction as follows.

In [4]:
using CondensedMatterSOS

bound, gram, ν = hamiltonian_energy(
    2,
    2,
    solver,
)
bound

Iter: 11 Ap: 9.59e-01 Pobj: -6.0000000e+00 Ad: 9.59e-01 Dobj: -6.0000000e+00 
Success: SDP solved
Primal objective value: -6.0000000e+00 
Dual objective value: -6.0000000e+00 
Relative primal infeasibility: 1.15e-15 
Relative dual infeasibility: 2.03e-09 
Real Relative Gap: 1.97e-09 
XZ Relative Gap: 4.05e-09 
DIMACS error measures: 1.72e-15 0.00e+00 3.15e-09 0.00e+00 1.97e-09 4.05e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -2.0855882e+01 Ad: 6.40e-01 Dobj: -4.8049171e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -1.8892667e+01 Ad: 9.56e-01 Dobj: -2.6648092e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -8.6626353e+00 Ad: 8.63e-01 Dobj: -4.4545586e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -6.4752587e+00 Ad: 8.01e-01 Dobj: -5.5908455e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -6.0344959e+00 Ad: 9.26e-01 Dobj: -5.9537973e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -6.0017524e+00 Ad: 9.65e-01 Dobj: -5.9975575e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -6.0000887e+00 Ad: 9.85e-0

-6.000000006574734

The reduction is obtained by block diagonalizing with a change of polynomial
basis to the isotypical basis.

In [5]:
display([M.basis.polynomials for M in ν.sub_moment_matrices])

[M.basis.polynomials for M in ν.sub_moment_matrices]

7-element Vector{Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}}:
 [(-0.7071067811865472 + 0.0im)σˣ₁ + (-0.7071067811865475 + 0.0im)σˣ₂]
 [(-0.7071067811865472 + 0.0im)σʸ₁ + (-0.7071067811865475 + 0.0im)σʸ₂]
 [(-0.7071067811865472 + 0.0im)σᶻ₁ + (-0.7071067811865475 + 0.0im)σᶻ₂]
 [(-0.7071067811865472 + 0.0im)σˣ₁ + (0.7071067811865475 + 0.0im)σˣ₂]
 [(-0.7071067811865472 + 0.0im)σʸ₁ + (0.7071067811865475 + 0.0im)σʸ₂]
 [(-0.7071067811865472 + 0.0im)σᶻ₁ + (0.7071067811865475 + 0.0im)σᶻ₂]
 [(1.0 - 0.0im)]

7-element Vector{Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}}:
 [(-0.7071067811865472 + 0.0im)σˣ₁ + (-0.7071067811865475 + 0.0im)σˣ₂]
 [(-0.7071067811865472 + 0.0im)σʸ₁ + (-0.7071067811865475 + 0.0im)σʸ₂]
 [(-0.7071067811865472 + 0.0im)σᶻ₁ + (-0.7071067811865475 + 0.0im)σᶻ₂]
 [(-0.7071067811865472 + 0.0im)σˣ₁ + (0.7071067811865475 + 0.0im)σˣ₂]
 [(-0.7071067811865472 + 0.0im)σʸ₁ + (0.7071067811865475 + 0.0im)σʸ₂]
 [(-0.7071067811865472 + 0.0im)σᶻ₁ + (0.7071067811865475 + 0.0im)σᶻ₂]
 [(1.0 - 0.0im)]

Let's try this for 3 sites. First without symmetry.

In [6]:
bound, gram, ν = hamiltonian_energy(
    3,
    2,
    solver,
    symmetry = false,
)
@show bound

Iter: 10 Ap: 9.69e-01 Pobj: -6.0000000e+00 Ad: 9.69e-01 Dobj: -6.0000000e+00 
Success: SDP solved
Primal objective value: -6.0000000e+00 
Dual objective value: -6.0000000e+00 
Relative primal infeasibility: 7.26e-17 
Relative dual infeasibility: 8.67e-10 
Real Relative Gap: 4.49e-10 
XZ Relative Gap: 1.18e-09 
DIMACS error measures: 1.08e-16 0.00e+00 2.09e-09 0.00e+00 4.49e-10 1.18e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -5.6096216e+01 Ad: 4.07e-01 Dobj: -4.5047371e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -5.5003096e+01 Ad: 9.56e-01 Dobj: -7.7051125e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -2.4831843e+01 Ad: 8.97e-01 Dobj: -1.2354955e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -6.5354954e+00 Ad: 8.45e-01 Dobj: -2.7588345e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -4.8443796e+00 Ad: 9.03e-01 Dobj: -4.2536148e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -4.5235956e+00 Ad: 9.40e-01 Dobj: -4.4765107e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -4.5013948e+00 Ad: 9.60e-0

-4.500000019875445

Now with symmetry.

In [7]:
bound, gram, ν = hamiltonian_energy(
    3,
    2,
    solver,
)
@show bound

Iter: 11 Ap: 9.58e-01 Pobj: -4.5000000e+00 Ad: 9.58e-01 Dobj: -4.5000000e+00 
Success: SDP solved
Primal objective value: -4.5000000e+00 
Dual objective value: -4.5000000e+00 
Relative primal infeasibility: 1.63e-16 
Relative dual infeasibility: 1.77e-09 
Real Relative Gap: 2.08e-09 
XZ Relative Gap: 3.52e-09 
DIMACS error measures: 3.26e-16 0.00e+00 4.63e-09 0.00e+00 2.08e-09 3.52e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -2.1488907e+01 Ad: 6.25e-01 Dobj: -3.4914792e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -1.9706704e+01 Ad: 9.49e-01 Dobj: -2.0607173e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -7.8323160e+00 Ad: 8.55e-01 Dobj: -3.2119246e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -5.0865942e+00 Ad: 8.14e-01 Dobj: -4.1692312e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -4.5468031e+00 Ad: 9.20e-01 Dobj: -4.4593892e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -4.5026279e+00 Ad: 9.54e-01 Dobj: -4.4973110e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -4.5001650e+00 Ad: 9.77e-0

-4.500000023057006

Let's look at the isotypical basis.

In [8]:
display([M.basis.polynomials for M in ν.sub_moment_matrices])

10-element Vector{Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}}:
 [(-0.5773502691896257 + 0.0im)σˣ₁ + (-0.5773502691896257 + 0.0im)σˣ₂ + (-0.5773502691896257 + 0.0im)σˣ₃]
 [(-0.5773502691896257 + 0.0im)σʸ₁ + (-0.5773502691896257 + 0.0im)σʸ₂ + (-0.5773502691896257 + 0.0im)σʸ₃]
 [(-0.5773502691896257 + 0.0im)σᶻ₁ + (-0.5773502691896257 + 0.0im)σᶻ₂ + (-0.5773502691896257 + 0.0im)σᶻ₃]
 [(-0.577350269189626 + 0.0im)σˣ₁ + (0.28867513459481275 + 0.5000000000000001im)σˣ₂ + (0.2886751345948131 - 0.49999999999999994im)σˣ₃]
 [(-0.577350269189626 + 0.0im)σʸ₁ + (0.28867513459481275 + 0.5000000000000001im)σʸ₂ + (0.2886751345948131 - 0.49999999999999994im)σʸ₃]
 [(-0.577350269189626 + 0.0im)σᶻ₁ + (0.28867513459481275 + 0.5000000000000001im)σᶻ₂ + (0.2886751345948131 - 0.49999999999999994im)σᶻ₃]
 [(-0.577350269189626 + 0.0im)σˣ₁ + (0.2886751345948131 - 0.49999999999999994im)σˣ₂ + (0.28867513459481275 + 0.5000000000000001im)σˣ₃]
 [(-0.577350269189626 + 0.0im)σʸ₁ + (0.2886751345948131 - 0.49999999

Now let's define a function for our common use case.

In [9]:
function f(L, d=1, consecutive=false; symmetry=true)
    @show L
    println("***")
    @show d
    bound, gram, ν = @time hamiltonian_energy(
        L,
        2d,
        solver,
        consecutive=consecutive,
        symmetry=symmetry,
    )
    @show bound
    for M in ν.sub_moment_matrices
        display(round.(M.basis.polynomials, digits=6))
    end
    println("E/N = ", bound / L)
    println("------------------------------------")
end

f(6, 1, true)

L = 6
***
d = 1
Iter: 10 Ap: 9.70e-01 Pobj: -4.5000000e+00 Ad: 9.70e-01 Dobj: -4.5000000e+00 
Success: SDP solved
Primal objective value: -4.5000000e+00 
Dual objective value: -4.5000000e+00 
Relative primal infeasibility: 5.49e-16 
Relative dual infeasibility: 2.47e-09 
Real Relative Gap: 2.08e-09 
XZ Relative Gap: 5.08e-09 
DIMACS error measures: 1.10e-15 0.00e+00 6.67e-09 0.00e+00 2.08e-09 5.08e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.56e-01 Pobj: -2.2102178e+01 Ad: 5.98e-01 Dobj: -7.0084924e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -3.3490766e+01 Ad: 9.50e-01 Dobj: -7.3463752e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -2.2209224e+01 Ad: 9.06e-01 Dobj: -1.2143640e+01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.9349852e+01 Ad: 7.85e-01 Dobj: -1.5786113e+01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.8132059e+01 Ad: 8.85e-01 Dobj: -1.7541366e+01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.8008840e+01 Ad: 9.33e-01 Dobj: -1.7949970e+01 
Iter:  7 Ap: 1.00e+00 Pobj: -1.8000695

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.408248 + 0.0im)σˣ₁ + (-0.408248 + 0.0im)σˣ₂ + (-0.408248 + 0.0im)σˣ₃ + (-0.408248 + 0.0im)σˣ₄ + (-0.408248 + 0.0im)σˣ₅ + (-0.408248 + 0.0im)σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.408248 + 0.0im)σʸ₁ + (-0.408248 + 0.0im)σʸ₂ + (-0.408248 + 0.0im)σʸ₃ + (-0.408248 + 0.0im)σʸ₄ + (-0.408248 + 0.0im)σʸ₅ + (-0.408248 + 0.0im)σʸ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.408248 + 0.0im)σᶻ₁ + (-0.408248 + 0.0im)σᶻ₂ + (-0.408248 + 0.0im)σᶻ₃ + (-0.408248 + 0.0im)σᶻ₄ + (-0.408248 + 0.0im)σᶻ₅ + (-0.408248 + 0.0im)σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.408248 + 0.0im)σˣ₁ + (0.204124 + 0.353553im)σˣ₂ + (0.204124 - 0.353553im)σˣ₃ + (-0.408248 + 0.0im)σˣ₄ + (0.204124 + 0.353553im)σˣ₅ + (0.204124 - 0.353553im)σˣ₆

...

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.408248 + 0.0im)σᶻ₁ + (-0.204124 - 0.353553im)σᶻ₂ + (0.204124 - 0.353553im)σᶻ₃ + (0.408248 + 0.0im)σᶻ₄ + (0.204124 + 0.353553im)σᶻ₅ + (-0.204124 + 0.353553im)σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.408248 + 0.0im)σˣ₁ + (0.408248 + 0.0im)σˣ₂ + (-0.408248 + 0.0im)σˣ₃ + (0.408248 + 0.0im)σˣ₄ + (-0.408248 + 0.0im)σˣ₅ + (0.408248 + 0.0im)σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.408248 + 0.0im)σʸ₁ + (0.408248 + 0.0im)σʸ₂ + (-0.408248 + 0.0im)σʸ₃ + (0.408248 + 0.0im)σʸ₄ + (-0.408248 + 0.0im)σʸ₅ + (0.408248 + 0.0im)σʸ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.408248 + 0.0im)σᶻ₁ + (0.408248 + 0.0im)σᶻ₂ + (-0.408248 + 0.0im)σᶻ₃ + (0.408248 + 0.0im)σᶻ₄ + (-0.408248 + 0.0im)σᶻ₅ + (0.408248 + 0.0im)σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (1.0 - 0.0im)

Now with `d = 2`.

In [10]:
f(6, 2, true)

L = 6
***
d = 2
Iter: 11 Ap: 9.59e-01 Pobj: -1.8000000e+01 Ad: 9.59e-01 Dobj: -1.8000000e+01 
Success: SDP solved
Primal objective value: -1.8000000e+01 
Dual objective value: -1.8000000e+01 
Relative primal infeasibility: 6.04e-15 
Relative dual infeasibility: 6.50e-10 
Real Relative Gap: 7.64e-10 
XZ Relative Gap: 1.16e-09 
DIMACS error measures: 1.58e-14 0.00e+00 2.22e-09 0.00e+00 7.64e-10 1.16e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.49e-01 Pobj: -2.5529703e+01 Ad: 5.17e-01 Dobj: -7.8034414e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -2.2424814e+02 Ad: 7.81e-01 Dobj:  1.1152854e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -2.0483924e+02 Ad: 7.58e-01 Dobj: -2.4950881e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -7.0145980e+01 Ad: 8.78e-01 Dobj: -2.7318044e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -2.8082270e+01 Ad: 8.25e-01 Dobj: -5.3450726e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -1.9399916e+01 Ad: 8.74e-01 Dobj: -9.6128294e+00 
Iter:  7 Ap: 8.31e-01 Pobj: -1.5297154

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.333333 + 0.0im)σˣ₁σˣ₂ + (-0.333333 + 0.0im)σˣ₁σˣ₆ + (0.166667 + 0.0im)σʸ₁σʸ₂ + (0.166667 + 0.0im)σʸ₁σʸ₆ + (0.166667 + 0.0im)σᶻ₁σᶻ₂ + (0.166667 + 0.0im)σᶻ₁σᶻ₆ + (-0.333333 + 0.0im)σˣ₂σˣ₃ + (0.166667 + 0.0im)σʸ₂σʸ₃ + (0.166667 + 0.0im)σᶻ₂σᶻ₃ + (-0.333333 + 0.0im)σˣ₃σˣ₄ + (0.166667 + 0.0im)σʸ₃σʸ₄ + (0.166667 + 0.0im)σᶻ₃σᶻ₄ + (-0.333333 + 0.0im)σˣ₄σˣ₅ + (0.166667 + 0.0im)σʸ₄σʸ₅ + (0.166667 + 0.0im)σᶻ₄σᶻ₅ + (-0.333333 + 0.0im)σˣ₅σˣ₆ + (0.166667 + 0.0im)σʸ₅σʸ₆ + (0.166667 + 0.0im)σᶻ₅σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.288675 + 0.0im)σʸ₁σʸ₂ + (-0.288675 + 0.0im)σʸ₁σʸ₆ + (0.288675 + 0.0im)σᶻ₁σᶻ₂ + (0.288675 + 0.0im)σᶻ₁σᶻ₆ + (-0.288675 + 0.0im)σʸ₂σʸ₃ + (0.288675 + 0.0im)σᶻ₂σᶻ₃ + (-0.288675 + 0.0im)σʸ₃σʸ₄ + (0.288675 + 0.0im)σᶻ₃σᶻ₄ + (-0.288675 + 0.0im)σʸ₄σʸ₅ + (0.288675 + 0.0im)σᶻ₄σᶻ₅ + (-0.288675 + 0.0im)σʸ₅σʸ₆ + (0.288675 + 0.0im)σᶻ₅σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.333333 + 0.0im)σˣ₁σˣ₂ + (0.166667 - 0.288675im)σˣ₁σˣ₆ + (0.166667 + 0.0im)σʸ₁σʸ₂ + (-0.083333 + 0.144338im)σʸ₁σʸ₆ + (0.166667 + 0.0im)σᶻ₁σᶻ₂ + (-0.083333 + 0.144338im)σᶻ₁σᶻ₆ + (0.166667 + 0.288675im)σˣ₂σˣ₃ + (-0.083333 - 0.144338im)σʸ₂σʸ₃ + (-0.083333 - 0.144338im)σᶻ₂σᶻ₃ + (0.166667 - 0.288675im)σˣ₃σˣ₄ + (-0.083333 + 0.144338im)σʸ₃σʸ₄ + (-0.083333 + 0.144338im)σᶻ₃σᶻ₄ + (-0.333333 + 0.0im)σˣ₄σˣ₅ + (0.166667 + 0.0im)σʸ₄σʸ₅ + (0.166667 + 0.0im)σᶻ₄σᶻ₅ + (0.166667 + 0.288675im)σˣ₅σˣ₆ + (-0.083333 - 0.144338im)σʸ₅σʸ₆ + (-0.083333 - 0.144338im)σᶻ₅σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.288675 + 0.0im)σʸ₁σʸ₂ + (0.144338 - 0.25im)σʸ₁σʸ₆ + (0.288675 + 0.0im)σᶻ₁σᶻ₂ + (-0.144338 + 0.25im)σᶻ₁σᶻ₆ + (0.144338 + 0.25im)σʸ₂σʸ₃ + (-0.144338 - 0.25im)σᶻ₂σᶻ₃ + (0.144338 - 0.25im)σʸ₃σʸ₄ + (-0.144338 + 0.25im)σᶻ₃σᶻ₄ + (-0.288675 + 0.0im)σʸ₄σʸ₅ + (0.288675 + 0.0im)σᶻ₄σᶻ₅ + (0.144338 + 0.25im)σʸ₅σʸ₆ + (-0.144338 - 0.25im)σᶻ₅σᶻ₆

...

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.288675 + 0.0im)σˣ₁σᶻ₂ + (0.144338 - 0.25im)σˣ₁σᶻ₆ + (0.288675 + 0.0im)σᶻ₁σˣ₂ + (-0.144338 + 0.25im)σᶻ₁σˣ₆ + (-0.144338 - 0.25im)σˣ₂σᶻ₃ + (0.144338 + 0.25im)σᶻ₂σˣ₃ + (0.144338 - 0.25im)σˣ₃σᶻ₄ + (-0.144338 + 0.25im)σᶻ₃σˣ₄ + (0.288675 + 0.0im)σˣ₄σᶻ₅ + (-0.288675 + 0.0im)σᶻ₄σˣ₅ + (0.144338 + 0.25im)σˣ₅σᶻ₆ + (-0.144338 - 0.25im)σᶻ₅σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.288675 + 0.0im)σʸ₁σᶻ₂ + (0.144338 - 0.25im)σʸ₁σᶻ₆ + (0.288675 + 0.0im)σᶻ₁σʸ₂ + (-0.144338 + 0.25im)σᶻ₁σʸ₆ + (-0.144338 - 0.25im)σʸ₂σᶻ₃ + (0.144338 + 0.25im)σᶻ₂σʸ₃ + (0.144338 - 0.25im)σʸ₃σᶻ₄ + (-0.144338 + 0.25im)σᶻ₃σʸ₄ + (0.288675 + 0.0im)σʸ₄σᶻ₅ + (-0.288675 + 0.0im)σᶻ₄σʸ₅ + (0.144338 + 0.25im)σʸ₅σᶻ₆ + (-0.144338 - 0.25im)σᶻ₅σʸ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.288675 + 0.0im)σˣ₁σʸ₂ + (-0.288675 + 0.0im)σˣ₁σʸ₆ + (0.288675 + 0.0im)σʸ₁σˣ₂ + (0.288675 + 0.0im)σʸ₁σˣ₆ + (0.288675 + 0.0im)σˣ₂σʸ₃ + (-0.288675 + 0.0im)σʸ₂σˣ₃ + (-0.288675 + 0.0im)σˣ₃σʸ₄ + (0.288675 + 0.0im)σʸ₃σˣ₄ + (0.288675 + 0.0im)σˣ₄σʸ₅ + (-0.288675 + 0.0im)σʸ₄σˣ₅ + (-0.288675 + 0.0im)σˣ₅σʸ₆ + (0.288675 + 0.0im)σʸ₅σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.288675 + 0.0im)σˣ₁σᶻ₂ + (-0.288675 + 0.0im)σˣ₁σᶻ₆ + (0.288675 + 0.0im)σᶻ₁σˣ₂ + (0.288675 + 0.0im)σᶻ₁σˣ₆ + (0.288675 + 0.0im)σˣ₂σᶻ₃ + (-0.288675 + 0.0im)σᶻ₂σˣ₃ + (-0.288675 + 0.0im)σˣ₃σᶻ₄ + (0.288675 + 0.0im)σᶻ₃σˣ₄ + (0.288675 + 0.0im)σˣ₄σᶻ₅ + (-0.288675 + 0.0im)σᶻ₄σˣ₅ + (-0.288675 + 0.0im)σˣ₅σᶻ₆ + (0.288675 + 0.0im)σᶻ₅σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (-0.288675 + 0.0im)σʸ₁σᶻ₂ + (-0.288675 + 0.0im)σʸ₁σᶻ₆ + (0.288675 + 0.0im)σᶻ₁σʸ₂ + (0.288675 + 0.0im)σᶻ₁σʸ₆ + (0.288675 + 0.0im)σʸ₂σᶻ₃ + (-0.288675 + 0.0im)σᶻ₂σʸ₃ + (-0.288675 + 0.0im)σʸ₃σᶻ₄ + (0.288675 + 0.0im)σᶻ₃σʸ₄ + (0.288675 + 0.0im)σʸ₄σᶻ₅ + (-0.288675 + 0.0im)σᶻ₄σʸ₅ + (-0.288675 + 0.0im)σʸ₅σᶻ₆ + (0.288675 + 0.0im)σᶻ₅σʸ₆

| id     | irep 1 | irep 2 | irep 3 | irep 4 |
|--------|--------|--------|--------|--------|
| degree | 2      | 3      | 1      | 3      |
| mult 2 | 1      | 3      | 2      | 1      |
| mult 3 | 3      | 6      | 4      | 3      |
| mult 4 | 6      | 10     | 7      | 6      |
| mult 5 | 10     | 15     | 11     | 10     |
| mult 6 | 15     | 21     | 16     | 15     |
| mult 7 | 21     | 28     | 22     | 21     |

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*